# Backend Target V2 API (EXAM CRITICAL!)

> **Exam Weight**: Part of 15% (Section 4) | **Must Master**: ✅✅

## 🧠 Conceptual Overview

### What is Backend V2 API?

The **Backend V2 API** is the modern interface for accessing quantum hardware information in Qiskit (2023+). It consolidates all hardware information into a single **`Target`** object.

### Architecture Comparison

```
V1 API (Legacy, Pre-2023):          V2 API (Current, 2023+):
┌──────────────────────────┐       ┌──────────────────────────┐
│ Backend                  │       │ Backend V2               │
│  ├─ configuration()      │       │  └─ target              │
│  ├─ properties()         │       │      ├─ operations       │
│  ├─ defaults()           │       │      ├─ qargs           │
│  └─ coupling_map         │       │      ├─ properties      │
└──────────────────────────┘       │      └─ instructions    │
Multiple scattered methods         └──────────────────────────┘
                                   Single unified interface
```

### Why V2 Matters for the Exam

- **V2 is the standard** for Qiskit 2.X certification
- Old V1 methods (`configuration()`, `properties()`) are deprecated
- Questions will test `backend.target` and `operation_names`

## Part 1: V2 vs V1 API

In [ ]:
# V2 vs V1 comparison
print("Backend API Versions:")
print("""
V1 Backend (OLD - DEPRECATED):
  backend.configuration()      # Returns BackendConfiguration
  backend.properties()         # Returns BackendProperties
  config.basis_gates           # List of supported gates
  
V2 Backend (NEW - STANDARD):
  backend.target               # Returns Target object
  backend.target.operation_names  # Replaces basis_gates
  backend.target.instruction_supported()  # Check gate availability
""")

print("\n🎯 EXAM TIP: V2 is the standard in Qiskit 2.X!")
print("✓ Use backend.target not configuration()")
print("✓ Use operation_names not basis_gates")

### ⚠️ EXAM TRAP: V2 API Changes

```python
# V1 (DEPRECATED)
config = backend.configuration()  # ❌
gates = config.basis_gates         # ❌

# V2 (CORRECT)
target = backend.target            # ✓
gates = target.operation_names     # ✓
```

## Part 2: Backend Target Basics

In [ ]:
# Target object
print("Backend Target:")
print("""
# Get target from backend
target = backend.target

# Get supported operations
ops = target.operation_names
print(ops)  # ['cx', 'rz', 'sx', 'x', 'id', 'measure', ...]

# Number of qubits
num_qubits = target.num_qubits
""")

print("\n✓ target is an attribute (not a method!)")
print("✓ operation_names includes measure, reset, etc.")
print("✓ More detailed than V1 basis_gates")

## Part 3: Operation Names vs Basis Gates

In [ ]:
# Operation names comparison
print("operation_names vs basis_gates:")
print("""
# V1 basis_gates (gates only)
['cx', 'rz', 'sx', 'x', 'id']

# V2 operation_names (all operations)
['cx', 'rz', 'sx', 'x', 'id', 'measure', 'reset', 'delay', ...]
""")

print("\nKey Differences:")
print("  V1: Only gate operations")
print("  V2: Gates + measure + reset + delay")
print("\n✓ V2 operation_names is more comprehensive")

### Common IBM Operation Names

| Operation | Type | Description |
|-----------|------|-------------|
| `cx` | Gate | CNOT (default entangling gate) |
| `ecr` | Gate | Echoed Cross-Resonance (some backends) |
| `rz` | Gate | Z-rotation (native parameterized) |
| `sx` | Gate | √X gate |
| `x` | Gate | Pauli X |
| `id` | Gate | Identity |
| `measure` | Operation | Measurement |
| `reset` | Operation | Qubit reset to |0⟩ |
| `delay` | Operation | Wait time (for timing) |

### V1 → V2 Translation Table (MEMORIZE!)

| **Feature** | **V1 API (Old)** | **V2 API (New)** |
|-------------|------------------|------------------|
| Get basis gates | `backend.configuration().basis_gates` | `backend.target.operation_names` |
| Get coupling map | `backend.configuration().coupling_map` | `backend.target.build_coupling_map()` |
| Check gate support | Manual parsing | `target.instruction_supported('cx', (0,1))` |
| Gate error rate | `backend.properties().gate_error(...)` | `target['cx'][(0,1)].error` |
| Number of qubits | `backend.configuration().n_qubits` | `backend.num_qubits` |

## Part 4: instruction_supported() (EXAM CRITICAL!)

In [ ]:
# Check instruction support
print("instruction_supported():")
print("""
# Check if gate supported on specific qubits
target = backend.target

# Check single-qubit gate
supported = target.instruction_supported('sx', qargs=(0,))
print(supported)  # True/False

# Check two-qubit gate
supported = target.instruction_supported('cx', qargs=(0, 1))
print(supported)  # True/False (depends on coupling map)
""")

print("\n✓ qargs is tuple of qubit indices")
print("✓ Returns True/False for gate availability")
print("✓ Respects coupling map constraints")

### ⚠️ EXAM TRAP: qargs Format

```python
# CORRECT - tuple
target.instruction_supported('cx', qargs=(0, 1))  # ✓
target.instruction_supported('sx', qargs=(0,))    # ✓

# WRONG - list or no comma for single qubit
target.instruction_supported('cx', qargs=[0, 1])  # ❌
target.instruction_supported('sx', qargs=(0))     # ❌
```

## Part 5: Target Properties

In [ ]:
# Target properties
print("Target Properties:")
print("""
target = backend.target

# Number of qubits
num_qubits = target.num_qubits

# All operations
operations = target.operation_names

# Get gate properties (error rates, duration)
props = target['cx'][(0, 1)]  # Properties for CX on qubits (0,1)
""")

print("\nTarget provides:")
print("  - Per-gate properties")
print("  - Per-qubit properties")
print("  - Coupling map information")
print("  - Error rates and durations")

## Part 6: V2 with Transpilation

In [ ]:
# Target with transpile
print("Target Integration:")
print("""
from qiskit import transpile

# V2 backend automatically uses target
transpiled = transpile(circuit, backend=backend)
# Internally uses backend.target for:
# - basis_gates (from operation_names)
# - coupling_map
# - gate properties

# Can also pass target explicitly
transpiled = transpile(circuit, target=backend.target)
""")

print("\n✓ transpile() auto-extracts from V2 backend")
print("✓ No need to manually specify constraints")
print("✓ Target contains all necessary information")

## 📝 Practice Questions

### Question 1: V2 API Access

**How to get supported operations in V2?**

A) `backend.configuration().basis_gates`  
B) `backend.target.operation_names`  
C) `backend.basis_gates`  
D) `backend.operations()`

<details>
<summary>Answer</summary>

**B) `backend.target.operation_names`**

```python
# V2 (CORRECT)
ops = backend.target.operation_names  # ✓

# V1 (DEPRECATED)
config = backend.configuration()      # ❌
gates = config.basis_gates            # ❌
```
</details>

---

### Question 2: Checking Gate Support

**How to check if CX supported on qubits (0,1)?**

A) `backend.target.has_gate('cx', (0,1))`  
B) `backend.target.instruction_supported('cx', qargs=(0,1))`  
C) `backend.supports('cx', [0,1])`  
D) `backend.can_execute('cx', 0, 1)`

<details>
<summary>Answer</summary>

**B) `backend.target.instruction_supported('cx', qargs=(0,1))`**

```python
supported = backend.target.instruction_supported(
    'cx', 
    qargs=(0, 1)  # Tuple!
)
```

**Note**: qargs must be tuple, not list!
</details>

---

### Question 3: V1 vs V2

**Which is V2 standard?**

A) `backend.configuration()`  
B) `backend.properties()`  
C) `backend.target`  
D) `backend.setup()`

<details>
<summary>Answer</summary>

**C) `backend.target`**

V2 uses Target object:
- `backend.target` (attribute, not method)
- `backend.target.operation_names`
- `backend.target.instruction_supported()`

V1 deprecated:
- `backend.configuration()`
- `backend.properties()`
</details>

---

### Question 4: Operation Names

**What does operation_names include that basis_gates doesn't?**

A) Nothing - they're identical  
B) measure, reset, delay  
C) Only measurement  
D) Custom gates

<details>
<summary>Answer</summary>

**B) measure, reset, delay**

```python
# V1 basis_gates (gates only)
['cx', 'rz', 'sx', 'x', 'id']

# V2 operation_names (all operations)
['cx', 'rz', 'sx', 'x', 'id', 'measure', 'reset', 'delay']
```

V2 includes non-gate operations!
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **V2 API (Standard)**
   - `backend.target` (attribute)
   - `backend.target.operation_names`
   - `backend.target.instruction_supported()`

2. **V1 API (Deprecated)**
   - `backend.configuration()`
   - `backend.properties()`
   - `config.basis_gates`

3. **Key Differences**
   - operation_names > basis_gates (includes measure, reset)
   - Target: per-qubit, per-gate properties
   - instruction_supported() checks specific qubits

### Critical Exam Facts

- ✅ V2 is Qiskit 2.X standard
- ✅ `backend.target` not `backend.configuration()`
- ✅ `operation_names` not `basis_gates`
- ✅ `instruction_supported(op, qargs=(i,j))`
- ✅ qargs must be tuple: `(0,)` not `(0)`
- ✅ operation_names includes measure/reset

### Common Traps

- ❌ `backend.configuration()` → ✅ `backend.target`
- ❌ `config.basis_gates` → ✅ `target.operation_names`
- ❌ `qargs=[0,1]` → ✅ `qargs=(0,1)`
- ❌ `qargs=(0)` → ✅ `qargs=(0,)`

### Mnemonic

🧠 **"V2 Target, not Configuration!"**

**Section 4 Complete!** ✅